In [1]:
import numpy as np
from pyfinite import ffield

In [2]:
expo = []
rang  =128
for i in range(rang):
    expo.append([-1]*128)

#a F_128 element will be represented as 7 bit integer i.e. x^2+x+1 = 0000111
F = ffield.FField(7, gen=0x83, useLUT=-1)

#functions for Field elements
def Add(x, y):
  ele1 = int(x)
  ele2 = int(y)
  a = x^y
  return a

def Multiply(ele1, ele2):
  pro = F.Multiply(ele1,ele2)
  return pro

def Exponentiate(ele, power):
    if(expo[ele][power]!=-1):
      return expo[ele][power]

    res = 0
    if(power==0):
      res = 1
    elif(power==1):
      res = ele
    elif(power%2 == 1):
      sqrt_ele = Exponentiate(ele,power//2)
      res = Multiply(ele,Multiply(sqrt_ele,sqrt_ele))

    else:
      res = Multiply(Exponentiate(ele,power//2), Exponentiate(ele,power//2))
    
    expo[ele][power] = res
    return res

def LinearTransform(matrix, ele_list):
  def addVectors(vec1, vec2):
    res=[0,0,0,0,0,0,0,0]
    for i, (ele1, ele2) in enumerate(zip(vec1, vec2)):
        res[i] = Add(ele1, ele2)
    return res

  def mulVectors(vec, ele):
    res= [0,0,0,0,0,0,0,0]
    for i, e in enumerate(vec):
      res[i] = Multiply(e,ele)
    return res
  
  res = [0,0,0,0,0,0,0,0]
  for row, ele in zip(matrix, ele_list):
    temp = mulVectors(row, ele)
    res= addVectors(temp, res)
  return res

In [3]:
#decodes full block of input cipher and return its hex
def decode_block(cipher):
  if(len(cipher)!=16):
      print("length isn't 16 for cipher %s" %cipher)
      exit(0)
  plain= ""
  for i in range(0,len(cipher),2):
      plain+=chr(16*(ord(cipher[i:i+2][0]) - ord('f')) + ord(cipher[i:i+2][1]) - ord('f'))
  return plain

In [4]:
#for diagonal elements
PossibleExponents = []
for i in range(8):
    PossibleExponents.append([])
    
possibleDiagonalVals=[]
for j in range(0,8):
    cur=[]
    for i in range(0,8):
        cur.append([])
    possibleDiagonalVals.append(cur)

with open("plaintexts.txt", 'r') as input_file, open("ciphertexts.txt", 'r') as output_file:
  for index, (input, output) in enumerate(zip(input_file.readlines(), output_file.readlines())):
      input_string = []
      output_string = []
      for msg in input.strip().split(" "):
        input_string.append(decode_block(msg)[index])
      for msg in output.strip().split(" "):
        output_string.append(decode_block(msg)[index])
      #input_string = [decode_block(msg)[index] for msg in input.strip().split(" ")]
      #output_string = [decode_block(msg)[index] for msg in output.strip().split(" ")]
      
      for i in range(1, 127):
        for j in range(1, 128):
          flag = True
          for inp, out in zip(input_string, output_string):
            if(ord(out) != Exponentiate(Multiply(Exponentiate(Multiply(Exponentiate(ord(inp), i), j), i), j), i)):
              flag = False
              break
          if(flag):
            PossibleExponents[index].append(i)
            possibleDiagonalVals[index][index].append(j)
print("Possible diagonal values: ")
print(possibleDiagonalVals)
print("Possible exponents: ")
print(PossibleExponents)

Possible diagonal values: 
[[[27, 84, 84], [], [], [], [], [], [], []], [[], [91, 70, 17], [], [], [], [], [], []], [[], [], [43, 14, 72], [], [], [], [], []], [[], [], [], [124, 12, 88], [], [], [], []], [[], [], [], [], [64, 100, 112], [], [], []], [[], [], [], [], [], [11, 73, 103], [], []], [[], [], [], [], [], [], [27, 66, 70], []], [[], [], [], [], [], [], [], [38, 61, 125]]]
Possible exponents: 
[[1, 19, 107], [13, 120, 121], [40, 89, 125], [51, 80, 123], [18, 21, 88], [52, 99, 103], [22, 37, 68], [15, 31, 81]]


In [5]:
with open("plaintexts.txt", 'r') as input_file, open("ciphertexts.txt", 'r') as output_file:
  for ind, (iline, oline) in enumerate(zip(input_file.readlines(), output_file.readlines())):
      if ind > 6 :
          break

      inpString = [decode_block(msg)[ind] for msg in iline.strip().split(" ")]
      outString = [decode_block(msg)[ind+1] for msg in oline.strip().split(" ")]

      for i in range(1, 128):
          for p1, e1 in zip(PossibleExponents[ind+1], possibleDiagonalVals[ind+1][ind+1]):
              for p2, e2 in zip(PossibleExponents[ind], possibleDiagonalVals[ind][ind]):
                  flag = True
                  for inp, outp in zip(inpString, outString):
                      if(ord(outp) != Exponentiate(Add(Multiply(Exponentiate(Multiply(Exponentiate(ord(inp), p2), e2), p2), i) ,Multiply(Exponentiate(Multiply(Exponentiate(ord(inp), p2), i), p1), e1)), p1)):
                          flag = False
                          break
                  if flag:
                      PossibleExponents[ind+1] = [p1]
                      possibleDiagonalVals[ind+1][ind+1] = [e1]
                      PossibleExponents[ind] = [p2]
                      possibleDiagonalVals[ind][ind] = [e2]
                      possibleDiagonalVals[ind][ind+1] = [i]
print(possibleDiagonalVals)
print(PossibleExponents)

[[[84], [115], [], [], [], [], [], []], [[], [70], [28], [], [], [], [], []], [[], [], [43], [24], [], [], [], []], [[], [], [], [12], [110], [], [], []], [[], [], [], [], [112], [111], [], []], [[], [], [], [], [], [11], [92], []], [[], [], [], [], [], [], [27], [3]], [[], [], [], [], [], [], [], [38]]]
[[19], [120], [40], [80], [88], [52], [22], [15]]


In [6]:
def EAEAE (plaintext, lin_mat, exp_mat): # Defines EAEAE
  plaintext = [ord(c) for c in plaintext]
  op = []
  for i in range(8):
    temp = []
    for j in range (8):
        temp.append(0)
    op.append(temp)

  #First Layer - Exponentiation
  for index, ele in enumerate(plaintext):
      op[0][index] = Exponentiate(ele, exp_mat[index])

  #Second Layer - Linear Transform
  op[1] = LinearTransform(lin_mat, op[0])

  #Third Layer - Exponentiation
  for index, ele in enumerate(op[1]):
      op[2][index] = Exponentiate(ele, exp_mat[index])

  #Fourth Layer - Linear Transform
  op[3] = LinearTransform(lin_mat, op[2])

  #Fifth Layer - Exponentiation
  for index, ele in enumerate(op[3]):
      op[4][index] = Exponentiate(ele, exp_mat[index])
      
  return op[4]

for indexex in range(0,6):
    offset = indexex + 2
    
    exp_list = []
    for e in PossibleExponents:
        exp_list.append(e[0])
    #exp_list = [e[0] for e in PossibleExponents]
    lin_trans_list = []
    
    for j in range(8):
        temp = []
        for i in range(8):
            temp.append(0)
        lin_trans_list.append(temp)
    #lin_trans_list = [[0 for i in range(8)] for j in range(8)]

    for i in range(8):
      for j in range(8):     
        if(len(possibleDiagonalVals[i][j]) != 0):
          lin_trans_list[i][j] = possibleDiagonalVals[i][j][0]
        else:
          lin_trans_list[i][j] = 0
          

    with open("plaintexts.txt", 'r') as input_file, open("ciphertexts.txt", 'r') as output_file:
      for index, (input, output) in enumerate(zip(input_file.readlines(), output_file.readlines())):
          if(index > (7-offset)):
            continue
          
          input_string = []
          output_string = []
          for msg in input.strip().split(" "):
            input_string.append(decode_block(msg))
          for msg in output.strip().split(" "):
            output_string.append(decode_block(msg))
          #input_string = [decode_block(msg) for msg in input.strip().split(" ")]
          #output_string = [decode_block(msg) for msg in output.strip().split(" ")]

          for i in range(1, 128):
              lin_trans_list[index][index+offset] = i
              flag = True
              for inps, outs in zip(input_string, output_string):
                  if EAEAE(inps, lin_trans_list, exp_list)[index+offset] != ord(outs[index+offset]):
                      flag = False
                      break
              if flag==True:
                  possibleDiagonalVals[index][index+offset] = [i]
    input_file.close();
    output_file.close();

lin_trans_list = []
for j in range(0,8):
    temp = []
    for i in range(0,8):
        temp.append(0)
    lin_trans_list.append(temp)
#lin_trans_list = [[0 for i in range(8)] for j in range(8)]
for i in range(0,8):
    for j in range(0,8):
      if len(possibleDiagonalVals[i][j]) == 0:
        lin_trans_list[i][j] = 0 
      else:
        lin_trans_list[i][j] = possibleDiagonalVals[i][j][0]

print(lin_trans_list)
print(exp_list)

[[84, 115, 8, 125, 97, 30, 17, 88], [0, 70, 28, 20, 43, 47, 120, 13], [0, 0, 43, 24, 3, 19, 18, 95], [0, 0, 0, 12, 110, 43, 101, 25], [0, 0, 0, 0, 112, 111, 2, 12], [0, 0, 0, 0, 0, 11, 92, 69], [0, 0, 0, 0, 0, 0, 27, 3], [0, 0, 0, 0, 0, 0, 0, 38]]
[19, 120, 40, 80, 88, 52, 22, 15]


In [7]:
#Final E and A as found above
At = [[84, 115, 8, 125, 97, 30, 17, 88], [0, 70, 28, 20, 43, 47, 120, 13], [0, 0, 43, 24, 3, 19, 18, 95], [0, 0, 0, 12, 110, 43, 101, 25], [0, 0, 0, 0, 112, 111, 2, 12], [0, 0, 0, 0, 0, 11, 92, 69], [0, 0, 0, 0, 0, 0, 27, 3], [0, 0, 0, 0, 0, 0, 0, 38]]
E = [19, 120, 40, 80, 88, 52, 22, 15]
print("E vector: ",E)
A = [[At[j][i] for j in range(len(At))] for i in range(len(At[0]))]
print("A matrix: ", A)

E vector:  [19, 120, 40, 80, 88, 52, 22, 15]
A matrix:  [[84, 0, 0, 0, 0, 0, 0, 0], [115, 70, 0, 0, 0, 0, 0, 0], [8, 28, 43, 0, 0, 0, 0, 0], [125, 20, 24, 12, 0, 0, 0, 0], [97, 43, 3, 110, 112, 0, 0, 0], [30, 47, 19, 43, 111, 11, 0, 0], [17, 120, 18, 101, 2, 92, 27, 0], [88, 13, 95, 25, 12, 69, 3, 38]]
